In [1]:
# from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
# import os
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import models,transforms,datasets
import torch
import bcolz
import time
%matplotlib inline

In [2]:
import imp
import utils; imp.reload(utils)
from utils import *

In [3]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

Using gpu: True 


### Load images

In [4]:
data_dir = '/home/foodlovers/FoodVisor/data/images'

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'test']}

In [7]:
dsets['train'].imgs[:5]

[('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_0.jpg', 0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_100.jpg',
  0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_101.jpg',
  0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_102.jpg',
  0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_104.jpg',
  0)]

In [9]:
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=32,
                                               shuffle=False, num_workers=6)
                for x in ['train', 'test']}

### Load VGG

In [10]:
model_vgg = models.vgg16(pretrained=True)
model_vgg.classifier._modules['6'] = nn.Linear(4096, 101)
if use_gpu:
    model_vgg = model_vgg.cuda()

### Calculate preconvoluted features

In [11]:
def preconvfeat(dataset):
    conv_features = []
    labels_list = []
    count = 1
    for data in dataset:
        print("{:d}/{:d}".format(count,len(dataset)),end="\r")
        count += 1
        inputs,labels = data
        if use_gpu:
            inputs , labels = Variable(inputs.cuda()),Variable(labels.cuda())
        else:
            inputs , labels = Variable(inputs),Variable(labels)
        
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list)

In [12]:
%%time
conv_feat_train,labels_train = preconvfeat(dset_loaders['train'])

/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 283
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlu

CPU times: user 19min, sys: 9min 45s, total: 28min 46s
Wall time: 28min 44s


In [13]:
%%time
conv_feat_val,labels_val = preconvfeat(dset_loaders['test'])

CPU times: user 5min 59s, sys: 3min 4s, total: 9min 4s
Wall time: 9min 4s


### Save features

In [15]:
save_array('/home/foodlovers/vgg16/conv_feat_train.bc',conv_feat_train)
save_array('/home/foodlovers/vgg16/labels_train.bc',labels_train)
save_array('/home/foodlovers/vgg16/conv_feat_val.bc',conv_feat_val)
save_array('/home/foodlovers/vgg16/labels_val.bc',labels_val)

NameError: name 'conv_feat_val' is not defined

### Load features

In [3]:
conv_feat_train = load_array('/home/foodlovers/vgg16/conv_feat_train.bc')
labels_train = load_array('/home/foodlovers/vgg16/labels_train.bc')
conv_feat_val = load_array('/home/foodlovers/vgg16/conv_feat_val.bc')
labels_val = load_array('/home/foodlovers/vgg16/labels_val.bc')

In [18]:
conv_feat_train.shape

(67988, 512, 7, 7)

In [19]:
len(dsets['train'].imgs) == len(conv_feat_train)

True

In [20]:
print(labels_train[:10])
print([x[1] for x in dsets['train'].imgs[:10]])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
for i,label in enumerate(labels_train) :
    if label != dsets['train'].imgs[i][1] :
        print("Wrong !!!")
for i,label in enumerate(labels_val) :
    if label != dsets['test'].imgs[i][1] :
        print("Wrong !!!")
print("True !")

True !


### Load corresponding text files

In [22]:
# Preprocessed textes in Food_Text_Classifier.ipynb
text_dir = '/home/foodlovers/FoodVisor/data/processed_texts_txt'

In [23]:
file_path = dsets['train'].imgs[1][0]
print(file_path.split("/")[-2:])
print("/".join(file_path.split("/")[-2:])[:-3]+'txt')
path_text = text_dir+'/'+"/".join(file_path.split("/")[-2:])[:-3]+'txt'
print(path_text)
text_file = open(path_text, "r")
text = text_file.read()
text_file.close()
print(text)

['apple_pie', 'apple_pie_100.jpg']
apple_pie/apple_pie_100.txt
/home/foodlovers/FoodVisor/data/processed_texts_txt/apple_pie/apple_pie_100.txt
My account My newslett My groceri list My save custom care log My box submit log subscrib digit edit give gift cookbook subscrib best BY cours appet breakfast dessert dinner side cours BY beef chicken strawberri tomato watermelon BY style grill ahead potluck quick slow cooker style healthi budget video partner advanc see submit contest save best deal subscrib today holiday holiday juli th labor day celebr summer entertain birthday holiday celebr fourth OF juli flagshap grill classic american pie red white blue dessert juli th commun commun favorit forum group commun facebook pinterest tumblr google twitter popular pin check pinterest follow school school Us find show school meet staff faq school find show attend show fun uniqu night learn skill memori 'll last lifetim find show cookbook across america discov countri best flavor shop cookbook acr

In [24]:
text_train = []
text_val = []
# a lot of file don't contain any text
valid_conv_feat_train = []
valid_labels_train = []
valid_conv_feat_val = []
valid_labels_val = []

unvalid_count = 0
for i in range(len(conv_feat_train)) :
    try :
        file_path = dsets['train'].imgs[i][0]
        path_text = text_dir+'/'+"/".join(file_path.split("/")[-2:])[:-3]+'txt'
        text_file = open(path_text, "r")
        text = text_file.read()
        text_file.close()
        text_train.append(text)
        valid_conv_feat_train.append(conv_feat_train[i])
        valid_labels_train.append(labels_train[i])
    except FileNotFoundError :
        # print(file_path)
        unvalid_count += 1
    
for i in range(len(conv_feat_val)) :
    try :
        file_path = dsets['test'].imgs[i][0]
        path_text = text_dir+'/'+"/".join(file_path.split("/")[-2:])[:-3]+'txt'
        text_file = open(path_text, "r")
        text = text_file.read()
        text_file.close()
        text_val.append(text)
        valid_conv_feat_val.append(conv_feat_val[i])
        valid_labels_val.append(labels_val[i])
    except FileNotFoundError :
        # print(file_path)
        unvalid_count += 1
        
print(unvalid_count," images don't have text file")

3908  images don't have text file


In [25]:
print("train:",len(text_train),"test:",len(text_val))

train: 65101 test: 21695


### Transform text file into TF-IDF

In [26]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(binary=True)
text_feat_train = tfidf.fit_transform(text_train)
#.astype('float16')
text_feat_val = tfidf.transform(text_val)

CPU times: user 57 s, sys: 319 ms, total: 57.3 s
Wall time: 56.3 s


In [27]:
text_feat_train.shape

(65101, 60033)

In [28]:
valid_conv_feat_train = np.asarray(valid_conv_feat_train).reshape(len(valid_conv_feat_train),-1)
valid_conv_feat_val = np.asarray(valid_conv_feat_val).reshape(len(valid_conv_feat_val),-1)

### Test image features with linearSVC

In [41]:
from sklearn.svm import LinearSVC

In [43]:
%%time
clf = LinearSVC()
clf.fit(valid_conv_feat_train, valid_labels_train)

CPU times: user 21min 36s, sys: 3.93 s, total: 21min 40s
Wall time: 21min 40s


In [45]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(valid_conv_feat_val)
accuracy = accuracy_score(valid_labels_val, y_pred)
print("Accuracy : ", accuracy)

Accuracy :  0.3752477529384651


## Build my model

Count les nombres de parametres ...

In [29]:
print("Text features shape:",text_feat_train.shape,"Image features shape:",valid_conv_feat_train.shape)

Text features shape: (65101, 60033) Image features shape: (65101, 25088)


In [30]:
use_gpu = torch.cuda.is_available()

### Train textual features with a sequential model

In [31]:
class myNet(nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(60033, 1024),
            nn.Linear(1024, 101)
        ) 

    def forward(self, x):
        x = self.classifier(x)
        return x

In [32]:
net1 = myNet()

if use_gpu:
    net1 = net1.cuda()

In [33]:
def data_gen_1(data,labels,batch_size=64,shuffle=True):
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(labels))
        data = data[index]
        labels = labels[index]
    for idx in range(0,len(labels),batch_size):
        if type(data).__module__ == 'numpy' :
            yield(data[idx:idx+batch_size],labels[idx:idx+batch_size],
             int(len(labels) / batch_size) + (len(labels) % batch_size > 0))
        else :
            yield(data[idx:idx+batch_size].toarray(),labels[idx:idx+batch_size],
             int(len(labels) / batch_size) + (len(labels) % batch_size > 0))

In [34]:
def train_model_1(model, criterion, batch_size = 64,
                 train_data = None, train_labels = None,
                 test_data = None, test_labels = None,
                  optimizer = None,
                 epochs = 1,train = True, validate = False,
                shuffle = True) :
    
    if train == True :
        loss_history = []
        acc_history = []
        val_loss_history = []
        val_acc_history = []
        
    for epoch in range(epochs) :
        if train == True :
            #=========================TRAINING=================================#
            start_time_epoch = time.time()
            
            model.train()
    
            print("Epoch:", epoch,"/",epochs-1,"===============================================")
        
            running_loss = 0.0
            running_corrects = 0.0
            
            batches = data_gen_1(data=train_data,labels=train_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
        
                inputs,classes,batch_num = data

                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                inputs = inputs.view(inputs.size(0), -1)
                    
                # calulate outputs and losses
                outputs = model(inputs)
                loss = criterion(outputs,classes)       

                # autograd
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # statistics
                batch_loss = loss.data.item()
                _,preds = torch.max(outputs.data,1)
                batch_corrects = torch.sum(preds == classes.data)
                
                running_loss += batch_loss
                running_corrects += batch_corrects

                print('Batch {:d}/{:d} - Loss: {:.4f} Acc: {:.4f} - Time : {:.2f}s'.format(i+1,batch_num,
                             batch_loss/len(classes), float(batch_corrects)/len(classes), time.time() - start_time),end='\r')

            epoch_loss = running_loss / len(train_labels)
            epoch_acc = running_corrects.data.item() / len(train_labels)
            #
            
            loss_history.append(epoch_loss)
            acc_history.append(epoch_acc)
            
            print('Epoch {:d} completed in {:.2f} seconds ! Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch , time.time() - start_time_epoch, epoch_loss, epoch_acc))
            
        if validate == True :
            #=========================VALIDATING=================================#
            
            model.eval()
            
            val_loss = 0.0
            val_corrects = 0.0
            
            batches = data_gen_1(data=test_data,labels=test_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
                
                inputs,classes,batch_num = data
                
                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()
                    
                inputs = inputs.view(inputs.size(0), -1)

                outputs = model(inputs)
                loss = criterion(outputs,classes)        
                _,preds = torch.max(outputs.data,1)

                # statistics

                val_loss += loss.data.item()
                val_corrects += torch.sum(preds == classes.data)
                
                print('Validating batch {:d}/{:d} - {:.2f}s ...'.format(i+1,batch_num
                                                                , time.time() - start_time), end="\r")

            val_epoch_loss = val_loss / len(test_labels)
            val_epoch_acc = val_corrects.data.item() / len(test_labels)
            # 

            print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                             val_epoch_loss,val_epoch_acc))
            
            if train == False :
                return
            else :
                val_loss_history.append(val_epoch_loss)
                val_acc_history.append(val_epoch_acc)
    
    if train == False :
        return 'On fait rien!'
    elif validate == False :
        return loss_history, acc_history
    else :
        return loss_history, acc_history,val_loss_history,val_acc_history

In [35]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
# optimizer = torch.optim.SGD(net1.parameters(),lr = lr,momentum = 0.9)
optimizer = torch.optim.Adam(net1.parameters())

In [36]:
%%time
history = train_model_1(net1,
                train_data=text_feat_train ,train_labels=valid_labels_train , 
                test_data = text_feat_val , test_labels = valid_labels_val ,
                epochs=5, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=True, validate = True,
                shuffle=True )
print("DONE !")

Epoch: 0 / 4 ===============================================
Epoch 0 completed in 69.15 seconds ! Loss: 0.0250 Acc: 0.6792
Val Loss: 0.0120 Val Acc: 0.8287 ...
Epoch: 1 / 4 ===============================================
Epoch 1 completed in 69.18 seconds ! Loss: 0.0053 Acc: 0.9277
Val Loss: 0.0116 Val Acc: 0.8341 ...
Epoch: 2 / 4 ===============================================
Epoch 2 completed in 69.30 seconds ! Loss: 0.0037 Acc: 0.9527
Val Loss: 0.0116 Val Acc: 0.8338 ...
Epoch: 3 / 4 ===============================================
Epoch 3 completed in 69.25 seconds ! Loss: 0.0033 Acc: 0.9590
Val Loss: 0.0116 Val Acc: 0.8364 ...
Epoch: 4 / 4 ===============================================
Epoch 4 completed in 69.18 seconds ! Loss: 0.0031 Acc: 0.9626
Val Loss: 0.0116 Val Acc: 0.8372 ...
DONE !
CPU times: user 5min 5s, sys: 1min 32s, total: 6min 38s
Wall time: 6min 35s


In [37]:
history = train_model_1(net1,
                train_data=text_feat_train ,train_labels=valid_labels_train , 
                test_data = text_feat_val , test_labels = valid_labels_val ,
                epochs=10, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=False, validate = True,
                shuffle=True )

Val Loss: 0.0116 Val Acc: 0.8372 ...


### Train on Image features

In [48]:
# if not defined above
model_vgg = models.vgg16(pretrained=True)
model_vgg.classifier._modules['6'] = nn.Linear(4096, 101)
if use_gpu:
    model_vgg = model_vgg.cuda()

In [38]:
optimizer = torch.optim.SGD(model_vgg.classifier.parameters(),lr = lr,momentum = 0.9)
# optimizer = torch.optim.Adam(model_vgg.classifier.parameters())

In [39]:
print(model_vgg.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=101, bias=True)
)


In [40]:
for param in model_vgg.classifier.parameters():
    param.requires_grad = True

In [41]:
%%time
history = train_model_1(model_vgg.classifier,
                train_data=valid_conv_feat_train  ,train_labels=valid_labels_train , 
                test_data = valid_conv_feat_val  , test_labels = valid_labels_val ,
                epochs=5, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=True, validate = True,
                shuffle=True )

Epoch: 0 / 4 ===============================================
Epoch 0 completed in 64.01 seconds ! Loss: 0.0504 Acc: 0.2562
Val Loss: 0.0413 Val Acc: 0.3900 ...
Epoch: 1 / 4 ===============================================
Epoch 1 completed in 112.47 seconds ! Loss: 0.0438 Acc: 0.3363
Val Loss: 0.0408 Val Acc: 0.4047 ...
Epoch: 2 / 4 ===============================================
Epoch 2 completed in 116.81 seconds ! Loss: 0.0390 Acc: 0.4010
Val Loss: 0.0406 Val Acc: 0.4126 ...
Epoch: 3 / 4 ===============================================
Epoch 3 completed in 116.54 seconds ! Loss: 0.0348 Acc: 0.4580
Val Loss: 0.0396 Val Acc: 0.4289 ...
Epoch: 4 / 4 ===============================================
Epoch 4 completed in 73.34 seconds ! Loss: 0.0309 Acc: 0.5150
Val Loss: 0.0394 Val Acc: 0.4374 ...
CPU times: user 5min 43s, sys: 2min 55s, total: 8min 38s
Wall time: 8min 35s


In [42]:
train_model_1(model_vgg.classifier,
                train_data=valid_conv_feat_train  ,train_labels=valid_labels_train , 
                test_data = valid_conv_feat_val  , test_labels = valid_labels_val ,
                epochs=5, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=True, validate = True,
                shuffle=True )

Epoch: 0 / 4 ===============================================
Epoch 0 completed in 112.31 seconds ! Loss: 0.0278 Acc: 0.5580
Val Loss: 0.0405 Val Acc: 0.4250 ...
Epoch: 1 / 4 ===============================================
Epoch 1 completed in 105.36 seconds ! Loss: 0.0255 Acc: 0.5953
Val Loss: 0.0407 Val Acc: 0.4287 ...
Epoch: 2 / 4 ===============================================


KeyboardInterrupt: 

In [43]:
train_model_1(model_vgg.classifier,
                train_data=valid_conv_feat_train  ,train_labels=valid_labels_train , 
                test_data = valid_conv_feat_val  , test_labels = valid_labels_val ,
                epochs=10, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=False, validate = True,
                shuffle=True )

Val Loss: 0.0401 Val Acc: 0.4276 ...


### Fusion

In [44]:
class myFusionNet(nn.Module):
    def __init__(self):
        super(myFusionNet, self).__init__()
#         self.fc1 = nn.Sequential(
#             nn.Linear(60033, 512),
#             nn.ReLU(True),
#             nn.Dropout(0.3),
#             nn.Linear(512, 128),
#             nn.ReLU(True),
#             nn.Dropout(0.3),
#             nn.Linear(128, 101),
#         ) 
#         self.fc2 = nn.Sequential(
#             nn.Linear(25088, 4096),
#             nn.ReLU(True),
#             nn.Dropout(0.3),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(0.5),
#             nn.Linear(4096, 101),
#         ) 
        self.ratio = torch.tensor(0.8, requires_grad=True, device="cuda")

    def forward(self, x):
        # x should have shape (?,60033+25088)
        x1 = x[:,:60033]
        x2 = x[:,-25088:]
        #x1 = self.fc1(x1)
        #x2 = self.fc2(x2)
        x1 = net1(x1)
        x2 = model_vgg.classifier(x2)
        out = self.ratio*x1 + (1-self.ratio)*x2
        # out = torch.cat((x1,x2),1)
        # out = self.classifier(out)
        return out

In [45]:
mynet = myFusionNet()
print(mynet)

myFusionNet()


In [46]:
if use_gpu:
    mynet = mynet.cuda()

### Loss & optimizer

In [47]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = torch.optim.SGD(list([mynet.ratio]),lr = lr,momentum = 0.9)

### Define datagen + train_model for data fusion

In [48]:
def data_gen(data1,data2,labels,batch_size=32,shuffle=True):
    """
    data1 expected to be text_feat of type csr_matrix
    data2 expected to be conv_feat of type ndarray
    """
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(labels))
        data1 = data1[index]
        data2 = data2[index]
        labels = labels[index]
    for idx in range(0,len(labels),batch_size):
        yield(np.concatenate((data1[idx:idx+batch_size].toarray(), data2[idx:idx+batch_size]), axis=1)
              ,labels[idx:idx+batch_size],
             int(len(labels) / batch_size) + (len(labels) % batch_size > 0))

In [49]:
def train_model_2(model, criterion,
                 train_data1 = None,train_data2=None, train_labels = None,
                 test_data1 = None,test_data2=None, test_labels = None,
                  optimizer = None,
                 epochs = 1,train = True, validate = False,
                shuffle = True) :
    
    if train == True :
        loss_history = []
        acc_history = []
        val_loss_history = []
        val_acc_history = []
        
    for epoch in range(epochs) :
        if train == True :
            #=========================TRAINING=================================#
            start_time_epoch = time.time()
            
            model.train()
    
            print("Epoch:", epoch,"/",epochs-1,"===============================================")
        
            running_loss = 0.0
            running_corrects = 0.0
            
            batches = data_gen(data1=train_data1,data2=train_data2,labels=train_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
        
                inputs,classes,batch_num = data

                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                inputs = inputs.view(inputs.size(0), -1)
                    
                # calulate outputs and losses
                outputs = model(inputs)
                loss = criterion(outputs,classes)       

                # autograd
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # statistics
                batch_loss = loss.data.item()
                _,preds = torch.max(outputs.data,1)
                batch_corrects = torch.sum(preds == classes.data)
                
                running_loss += batch_loss
                running_corrects += batch_corrects

                print('Batch {:d}/{:d} - Loss: {:.4f} Acc: {:.4f} - Time : {:.2f}s'.format(i+1,batch_num,
                             batch_loss/len(classes), float(batch_corrects)/len(classes), time.time() - start_time),end='\r')

            epoch_loss = running_loss / len(train_labels)
            epoch_acc = running_corrects.data.item() / len(train_labels)
            #
            
            loss_history.append(epoch_loss)
            acc_history.append(epoch_acc)
            
            print('Epoch {:d} completed in {:.2f} seconds ! Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch , time.time() - start_time_epoch, epoch_loss, epoch_acc))
            
        if validate == True :
            #=========================VALIDATING=================================#
            
            model.eval()
            
            val_loss = 0.0
            val_corrects = 0.0
            
            batches = data_gen(data1=test_data1,data2=test_data2,labels=test_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
                
                inputs,classes,batch_num = data
                
                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()
                    
                inputs = inputs.view(inputs.size(0), -1)

                outputs = model(inputs)
                loss = criterion(outputs,classes)        
                _,preds = torch.max(outputs.data,1)

                # statistics

                val_loss += loss.data.item()
                val_corrects += torch.sum(preds == classes.data)
                
                print('Validating batch {:d}/{:d} - {:.2f}s ...'.format(i+1,batch_num
                                                                , time.time() - start_time), end="\r")

            val_epoch_loss = val_loss / len(test_labels)
            val_epoch_acc = val_corrects.data.item() / len(test_labels)
            # 

            print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                             val_epoch_loss,val_epoch_acc))
            
            if train == False :
                return
            else :
                val_loss_history.append(val_epoch_loss)
                val_acc_history.append(val_epoch_acc)
    
    if train == False :
        return 'On fait rien!'
    elif validate == False :
        return loss_history, acc_history
    else :
        return loss_history, acc_history,val_loss_history,val_acc_history

### Training

In [50]:
history = train_model_2(model=mynet,criterion=criterion,
              train_data1 = text_feat_train,train_data2=valid_conv_feat_train, train_labels = valid_labels_train,
                 test_data1 = text_feat_val,test_data2= valid_conv_feat_val, test_labels = valid_labels_val,
                  optimizer = optimizer,
                 epochs = 5,train = True, validate = True,
                shuffle = True)

Epoch: 0 / 4 ===============================================
Epoch 0 completed in 184.35 seconds ! Loss: 0.0025 Acc: 0.9882
Val Loss: 0.0164 Val Acc: 0.8817 ...
Epoch: 1 / 4 ===============================================
Epoch 1 completed in 182.49 seconds ! Loss: 0.0025 Acc: 0.9884
Val Loss: 0.0164 Val Acc: 0.8834 ...
Epoch: 2 / 4 ===============================================
Epoch 2 completed in 182.68 seconds ! Loss: 0.0025 Acc: 0.9881
Val Loss: 0.0164 Val Acc: 0.8836 ...
Epoch: 3 / 4 ===============================================


KeyboardInterrupt: 

In [51]:
mynet.ratio

tensor(0.5331, device='cuda:0', requires_grad=True)

In [52]:
history = train_model_2(model=mynet,criterion=criterion,
              train_data1 = text_feat_train,train_data2=valid_conv_feat_train, train_labels = valid_labels_train,
                 test_data1 = text_feat_val,test_data2= valid_conv_feat_val, test_labels = valid_labels_val,
                  optimizer = optimizer,
                 epochs = 10,train = False, validate = True,
                shuffle = True)

Val Loss: 0.0170 Val Acc: 0.8836 ...
